In [2]:
# CELL 1: Full corrected code, now importing the missing functions

from ipyleaflet import Map, basemaps, Marker, GeoJSON, Popup
from IPython.display import display, Markdown
import requests
import json

# PyNHD import for flowlines and streamgages
from pynhd import NLDI

# Import streamgage popup module
from gage_popup_utils import create_enhanced_popup_content

# Import StreamCat reporting
from streamcat_data import display_streamcat_report_for_comid

# Import the functions that were previously in cell 3
# (adjust the file name and import statement to match your setup)
from watershed_utils import format_watershed_report, display_map_layers

# 1. Create Interactive Map
center_usa = (47.897, -123.111)  # Approximate center of USA
zoom_level = 9

usa_map = Map(center=center_usa, zoom=zoom_level, basemap=basemaps.Esri.WorldStreetMap)
display(usa_map)

def handle_map_click(event, **kwargs):
    """Handles map click, retrieves COMID from NLDI,
    fetches Watershed Report, and displays formatted report and map layers.
    Afterward, it also displays the StreamCat report."""
    if kwargs.get('type') == 'click':
        latlng = kwargs.get('coordinates')
        latitude = latlng[0]
        longitude = latlng[1]

        # NLDI API URL for COMID
        nldi_url = f"https://api.water.usgs.gov/nldi/linked-data/comid/position?coords=POINT({longitude} {latitude})"
        api_key = 'grRcWoOIWpPgTCG40v4MFRxR1MtxPfNOrkltHvY8'  # Your API key

        try:
            response_nldi = requests.get(url=nldi_url, headers={'accept': 'application/json'})
            response_nldi.raise_for_status()
            nldi_data = response_nldi.json()
            comid = None
            features = nldi_data.get('features', [])
            if features:
                properties = features[0].get('properties', {})
                comid = properties.get('comid')

            if comid:
                print(f"Clicked at Latitude: {latitude}, Longitude: {longitude}")
                print(f"NLDI COMID: {comid}")

                # --- EPA WATERS API Request (Watershed Report v2.5) ---
                waters_api_url = f"https://api.epa.gov/waters/v2_5/nhdplus_json?pcomid={comid}&f=json"
                headers = {'X-Api-Key': api_key}
                try:
                    # Debug prints
                    print(f"[DEBUG] Requesting WATERS at {waters_api_url} ...")
                    response_waters = requests.get(waters_api_url, headers=headers)
                    print(f"[DEBUG] WATERS response status code: {response_waters.status_code}")
                    response_waters.raise_for_status()

                    waters_data = response_waters.json()
                    # Debug info
                    print(f"[DEBUG] WATERS response keys: {list(waters_data.keys())}")

                    # Format and Display the Watershed Report
                    formatted_report = format_watershed_report(waters_data)
                    display(formatted_report)

                    # --- Fetch and display map layers ---
                    display_map_layers(comid, usa_map)

                    # --- Display the StreamCat report for this COMID ---
                    display_streamcat_report_for_comid(comid)

                except requests.exceptions.RequestException as e_waters:
                    print(f"Error: Failed to connect to EPA WATERS API (v2.5). {e_waters}")
                except json.JSONDecodeError:
                    print("Error: Could not decode JSON response from EPA WATERS API (v2.5).")

            else:
                print("Error: COMID not found in NLDI response. Cannot proceed to WATERS API.")

        except requests.exceptions.RequestException as e_nldi:
            print(f"Error: Failed to connect to NLDI API. {e_nldi}")
        except json.JSONDecodeError:
            print("Error: Could not decode JSON response from NLDI API.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

usa_map.on_interaction(handle_map_click)

print("Click on the map to get NLDI COMID, Watershed data (v2.5), Streamgage popup info, and StreamCat report!")

Map(center=[47.897, -123.111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Click on the map to get NLDI COMID, Watershed data (v2.5), Streamgage popup info, and StreamCat report!


Clicked at Latitude: 47.95498440806741, Longitude: -123.10180664062501
NLDI COMID: 23997456
[DEBUG] Requesting WATERS at https://api.epa.gov/waters/v2_5/nhdplus_json?pcomid=23997456&f=json ...
[DEBUG] WATERS response status code: 200
[DEBUG] WATERS response keys: ['output', 'status']


### For the stream segment
| Attribute | Value | Unit |
|---|---|---|
| Stream Name | Dungeness River | None |
| Stream Order | 3 | None |
| Stream Level | 1 | None |
| Mean annual flow volume (estimate) | 283.521 |  cfs |
| Mean annual flow velocity (estimate) | 1.7922 |  fps |
| Stream Length | 2.74932423881151 |  km |
| Stream Time of Travel (estimate) | 0.05824513 |  days |

#### Category: For the catchment (local area draining directly to the selected stream segment)
##### Subcategory: Metrics
| Attribute | Value | Unit |
|---|---|---|
| Catchment area measurement | 4.73759379263287 |  km² |
| Mean annual temperature | 8.76124023 |  °C |
| Mean annual precipitation | 1351.49 |  mm |

##### Subcategory: 2011 National Land Cover Dataset
| Attribute | Value | Unit |
|---|---|---|
| Open Water (11) | 0 | % |
| Low Intensity Residential (21) | 0 | % |
| Commercial (23) | 0 | % |
| Deciduous Forest (41) | 0.721473 | % |
| Evergreen Forest (42) | 96.601481 | % |
| Mixed Forest (43) | 1.499905 | % |
| Other | 1.177141 | % |

#### Category: For the watershed (drainage area extending from the outlet upstream to the headwaters)
##### Subcategory: Metrics
| Attribute | Value | Unit |
|---|---|---|
| Drainage area measurement | 178.3188 |  km² |
| Mean annual temperature | 5.18039817 |  °C |
| Mean annual precipitation | 1983.2151927 |  mm |

##### Subcategory: 2011 National Land Cover Dataset
| Attribute | Value | Unit |
|---|---|---|
| Open Water (11) | 0.026245 | % |
| Low Intensity Residential (21) | 0.323017 | % |
| Commercial (23) | 0 | % |
| Deciduous Forest (41) | 0.090344 | % |
| Evergreen Forest (42) | 70.683181 | % |
| Mixed Forest (43) | 0.294753 | % |
| Other | 28.58246 | % |



### StreamCat Data Report

#### Landscape Layer: 2006 National Land Cover Database Agricultural Land Cover on Slopes
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent agricultural land cover (classes 81 and 82) of the catchment area classified by the NLCD2006 landscape raster where slope is greater than 10%. | 0% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the watershed area classified by the NLCD2006 landscape raster where slope is greater than 10%. | .0171602769870591% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the catchment area classified by the NLCD2006 landscape raster where slope is equal to or greater than 20%. | 0% | 100 |
| Percent agricultural land cover (classes 81 and 82) of the watershed area classified by the NLCD2006 landscape raster where slope is equal to or greater than 20%. | .0171602769870591% | 100 |

#### Landscape Layer: Agricultural Nitrogen Inputs
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean rate of biological nitrogen fixation from the cultivation of crops in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of biological nitrogen fixation from the cultivation of crops in kg N/ha/yr within the total upstream watershed. | .00257404154805887kg/ha/yr | 100 |
| Mean rate of synthetic nitrogen fertilizer application in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of synthetic nitrogen fertilizer application in kg N/ha/yr within the total upstream watershed. | .00531968586598833kg/ha/yr | 100 |
| Mean rate of manure application from confined animal feeding operations in kg N/ha/yr within the local catchment. | 0kg/ha/yr | 100 |
| Mean rate of manure application from confined animal feeding operations in kg N/ha/yr within the total upstream watershed. | .000686411079482365kg/ha/yr | 100 |

#### Landscape Layer: Base Flow Index
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Ratio of base flow to total flow, expressed as a percentage within the local catchment. | 59.0322764382001% | 100 |
| Ratio of base flow to total flow, expressed as a percentage within the total upstream watershed. | 58.6511063331516% | 100 |

#### Landscape Layer: Canal Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of NHDPlusV2 line features within the local catchment classified as canal, ditch, or pipeline.(kilometer of canal/square kilometer). | 0km/km² | 100 |
| Density of NHDPlusV2 line features within the total upstream watershed classified as canal, ditch, or pipeline.(kilometer of canal/square kilometer). | 0km/km² | 100 |

#### Landscape Layer: National Coal Resource Dataset System
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced coal mine sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced coal mine sites within the total upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: Dam Density and Storage Volume
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced dams within the local catchment. | 0dams/km² | 100 |
| Density of georeferenced dams within the upstream watershed. | 0dams/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the local catchment. | 0m³/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the total upstream watershed. | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the local catchment. | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the total upstream watershed. | 0m³/km² | 100 |

#### Landscape Layer: National Elevation Dataset
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all elev_cm values within the local catchment divided by 100 to convert cm to m. | 614.25386368m | 100 |
| Mean of all elev_cm values within the upstream watershed divided by 100 to convert cm to m. | 1311.42113081m | 100 |

#### Landscape Layer: Facility Registry Services (FRS): Toxic Release Inventory (TRI), National Pollutant Discharge Elimination System (NPDES), and Superfund Sites
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced National Pollutant Discharge Elimination System sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced National Pollutant Discharge Elimination System sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Superfund sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced Toxic Release Inventory sites within the riparian mask of the upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: Wildland Fire Perimeters By Year 2000 - 2010
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage forest loss to fire (fire perimeter) in 2000 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2000 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2001 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2002 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2003 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2004 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2005 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2006 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2007 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2008 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2009 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the total upstream watershed. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the riparian mask of the local catchment. | 0% | 100 |
| Percentage forest loss to fire (fire perimeter) in 2010 within the riparian mask of the total upstream watershed. | 0% | 100 |

#### Landscape Layer: Forest Loss By Year 2001 to 2013
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent of forest loss detected primarily in the year 2001 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2001 within the total upstream watershed. | .00151414208709% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the total upstream watershed. | .00302828417419% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the local catchment. | .0379722802354% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the total upstream watershed. | .00908485252256% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the total upstream watershed. | .00201885611612% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the total upstream watershed. | .00151414208709% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the total upstream watershed. | .00454242626128% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the total upstream watershed. | .0509761169321% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the total upstream watershed. | .00252357014516% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the total upstream watershed. | .00555185431934% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the total upstream watershed. | .0156461349% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the total upstream watershed. | .016655562958% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the total upstream watershed. | .00100942805806% | 100 |

#### Landscape Layer: Riparian Buffer Forest Loss By Year 2001 to 2013
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percent of forest loss detected primarily in the year 2001 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2001 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2002 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the riparian mask of the local catchment. | .151745068285281% | 100 |
| Percent of forest loss detected primarily in the year 2003 within the riparian mask of the total upstream watershed. | .00423818605636787% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2004 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2005 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2006 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2007 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2008 within the riparian mask of the total upstream watershed. | .0720491629582539% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2009 within the riparian mask of the total upstream watershed. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2010 within the riparian mask of the total upstream watershed. | .0296673023945751% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2011 within the riparian mask of the total upstream watershed. | .00847637211273575% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2012 within the riparian mask of the total upstream watershed. | .033905488450943% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the riparian mask of the local catchment. | 0% | 100 |
| Percent of forest loss detected primarily in the year 2013 within the riparian mask of the total upstream watershed. | 0% | 100 |

#### Landscape Layer: Olson and Hawkins (2014) Geochemical and Geophysical Characteristics
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Lithological aluminum oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 12.8344087550028% | 100 |
| Lithological aluminum oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 11.8253675196334% | 100 |
| Lithological calcium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 5.15521359637365% | 100 |
| Lithological calcium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 5.93153515822785% | 100 |
| Lithological ferric oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 6.95845223495348% | 100 |
| Lithological ferric oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 6.51525654174994% | 100 |
| Lithological potassium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 1.48764812380482% | 100 |
| Lithological potassium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 1.44008116228575% | 100 |
| Lithological magnesium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 3.8894859558952% | 100 |
| Lithological magnesium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 3.72288088982093% | 100 |
| Lithological sodium oxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 1.79497480324473% | 100 |
| Lithological sodium oxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 1.72963544178124% | 100 |
| Lithological phosphorus pentoxide content in surface or near surface geology, expressed as a percentage within the local catchment. | .229366974609835% | 100 |
| Lithological phosphorus pentoxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .21307133566814% | 100 |
| Lithological sulfur content in surface or near surface geology, expressed as a percentage within the local catchment. | .0741475622642871% | 100 |
| Lithological sulfur content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .0361916858310621% | 100 |
| Lithological silicon dioxide content in surface or near surface geology, expressed as a percentage within the local catchment. | 59.7335541729637% | 100 |
| Lithological silicon dioxide content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | 61.1035409630953% | 100 |
| Lithological nitrogen content in surface or near surface geology, expressed as a percentage within the local catchment. | .041191507649326% | 100 |
| Lithological nitrogen content in surface or near surface geology, expressed as a percentage within the total upstream watershed. | .0222128799754204% | 100 |
| Hydraulic conductivity (in micrometers per second) of surface or near surface geology, within the local catchment. | .381072371870135% | 100 |
| Hydraulic conductivity (in micrometers per second) of surface or near surface geology, within the total upstream watershed. | .419467569608645% | 100 |
| Compressive strength, measured as uniaxial compressive strength (in megaPascals) of surface or near surface geology, within the local catchment | 127.716145870515% | 100 |
| Compressive strength, measured as uniaxial compressive strength (in megaPascals) of surface or near surface geology, within the total upstream watershed | 137.326406346779% | 100 |

#### Landscape Layer: 2006 National Land Cover Database Impervious Surfaces
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean imperviousness of anthropogenic surfaces within catchment. | 0% | 100 |
| Mean imperviousness of anthropogenic surfaces within watershed. | .0358094603597602% | 100 |
| Mean imperviousness of anthropogenic surfaces within catchment and within a 100-m buffer of NHD stream lines. | 0% | 100 |
| Mean imperviousness of anthropogenic surfaces within watershed and within a 100-m buffer of NHD stream lines. | .04089849544395% | 100 |

#### Landscape Layer: Soil Erodibility (KFFACT)
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean agricultural surface soil erodibility factor adjusted within the local catchment. | 0 | 100 |
| Mean agricultural surface soil erodibility factor adjusted within the total upstream watershed. | .0000463327478650597 | 100 |
| Mean surface soil erodibility factor adjusted within the local catchment. | .27 | 100 |
| Mean surface soil erodibility factor adjusted within the total upstream watershed. | .216177750186744 | 100 |

#### Landscape Layer: Surficial Lithology
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of catchment area classified as lithology type: alkaline intrusive volcanic rock. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: alkaline intrusive volcanic rock. | 0% | 100 |
| Percentage of catchment area classified as lithology type: alluvium and fine-textured coastal zone sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: alluvium and fine-textured coastal zone sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: carbonate residual material. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: carbonate residual material. | 0% | 100 |
| Percentage of catchment area classified as lithology type: coastal zone sediment, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: coastal zone sediment, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: colluvial sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: colluvial sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: eolian sediment, coarse-textured (sand dunes). | 0% | 100 |
| Percentage of watershed area classified as as lithology type: eolian sediment, coarse-textured (sand dunes). | 0% | 100 |
| Percentage of catchment area classified as lithology type: eolian sediment, fine-textured (glacial loess). | 0% | 100 |
| Percentage of watershed area classified as as lithology type: eolian sediment, fine-textured (glacial loess). | 0% | 100 |
| Percentage of catchment area classified as lithology type: extrusive volcanic rock. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: extrusive volcanic rock. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial outwash and glacial lake sediment, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial outwash and glacial lake sediment, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial lake sediment, fine-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial lake sediment, fine-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, clayey. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, clayey. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, coarse-textured. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, coarse-textured. | 0% | 100 |
| Percentage of catchment area classified as lithology type: glacial till, loamy. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: glacial till, loamy. | 0% | 100 |
| Percentage of catchment area classified as lithology type: hydric, peat and muck. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: hydric, peat and muck. | 0% | 100 |
| Percentage of catchment area classified as lithology type: non-carbonate residual material. | 46.0603759255743% | 100 |
| Percentage of watershed area classified as as lithology type: non-carbonate residual material. | 55.9021258554903% | 100 |
| Percentage of catchment area classified as lithology type: saline like sediment. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: saline like sediment. | 0% | 100 |
| Percentage of catchment area classified as lithology type: silicic residual material. | 53.9396240744257% | 100 |
| Percentage of watershed area classified as as lithology type: silicic residual material. | 44.0978741445097% | 100 |
| Percentage of catchment area classified as lithology type: water. | 0% | 100 |
| Percentage of watershed area classified as as lithology type: water | 0% | 100 |

#### Landscape Layer: Mine Density Active Mines and Mineral Plants in the US
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced mines and mineral plants within the local catchment. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the upstream watershed. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the riparian mask of the local catchment. | 0sites/km² | 100 |
| Density of georeferenced mines and mineral plants within the riparian mask of the upstream watershed. | 0sites/km² | 100 |

#### Landscape Layer: National Anthropenic Barrier Dataset
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Density of georeferenced dams within the local catchment (dams/square km). | 0dams/km² | 100 |
| Density of georeferenced dams within the total upstream watershed (dams/square km). | 0dams/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the local catchment (cubic meters/square km). | 0m³/km² | 100 |
| Mean NID storage volume of all dam reservoirs (NID_STORA in NID) within the total upstream watershed (cubic meters/square km). | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the local catchment (cubic meters/square km). | 0m³/km² | 100 |
| Mean normal storage volume of all dam reservoirs (NORM_STORA in NID) within the total upstream watershed (cubic meters/square km). | 0m³/km² | 100 |

#### Landscape Layer: National Atmospheric Deposition Program National Trends Network - Nitrogen Deposition
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Annual average of precipitation-weighted mean wet deposition for inorganic nitrogen concentration from nitrate and ammonium for year, 2008, within the local catchment. | .409684543713689kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean wet deposition for inorganic nitrogen concentration from nitrate and ammonium for year, 2008, within the upstream watershed. | .596941921890457m³/km² | 100 |
| Annual average of precipitation-weighted mean deposition for ammonium ion concentration for year, 2008, within the local catchment. | .231720124708563kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for ammonium ion concentration for year, 2008, within the total upstream watershed. | .333735392231442kg/ha/yr | 100 |
| Annual average of preciptiation-weighted mean deposition for nitrate ion concentration for year, 2008, within the local catchment. | 1.00385248378204kg/ha/yr | 100 |
| Annual average of preciptiation-weighted mean deposition for nitrate ion concentration for year, 2008, within the total upstream watershed. | 1.47532729169947kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for average sulfur and nitrogen concentration for year, 2008, within the local catchment. | 81.3131739447503kg/ha/yr | 100 |
| Annual average of precipitation-weighted mean deposition for average sulfur and nitrogen concentration for year, 2008, within the total upstream watershed. | 120.550164340944kg/ha/yr | 100 |

#### Landscape Layer: 2006 National Land Cover Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of the local catchment area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 0% | 100 |
| Percentage of the upstream watershed area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 13.6535239133507% | 100 |
| Percentage of the local catchment classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 3.02182665819029% | 100 |
| Percentage of the local catchment area classified as evergreen forest land cover. | 96.5635086386938% | 100 |
| Percentage of the upstream watershed area classified as evergreen forest land cover. | 70.6761149132901% | 100 |
| Percentage of the local catchment classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 91.9575113808801% | 100 |
| Percentage of the upstream watershed classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 86.9718160627252% | 100 |
| Percentage of the local catchment area classified as row crop land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as row crop land use. | 0% | 100 |
| Percentage of the local catchment classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as deciduous forest land cover. | .113916840706284% | 100 |
| Percentage of the upstream watershed area classified as deciduous forest land cover. | .0741929622675792% | 100 |
| Percentage of the local catchment classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | .910470409711684% | 100 |
| Percentage of the upstream watershed classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | .186480186480186% | 100 |
| Percentage of the local catchment area classified as grassland/herbaceous land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as grassland/herbaceous land cover. | 3.92213271960108% | 100 |
| Percentage of the local catchment classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | 1.03411739775376% | 100 |
| Percentage of the local catchment area classified as pasture/hay land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as pasture/hay land use. | .0171602769870591% | 100 |
| Percentage of the local catchment classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as herbaceous wetland land cover. | .00252357014515576% | 100 |
| Percentage of the local catchment classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as ice/snow land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as ice/snow land cover. | 3.25843377142511% | 100 |
| Percentage of the local catchment classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 1.6486543759271% | 100 |
| Percentage of the local catchment area classified as mixed deciduous/evergreen forest land cover. | 1.38598822859313% | 100 |
| Percentage of the upstream watershed area classified as mixed deciduous/evergreen forest land cover. | .291724708780005% | 100 |
| Percentage of the local catchment classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 1.06221547799697% | 100 |
| Percentage of the upstream watershed classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | .593346047891502% | 100 |
| Percentage of the local catchment area classified as open water land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | .0262451295096199% | 100 |
| Percentage of the local catchment classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | .0254291163382072% | 100 |
| Percentage of the local catchment area classified as shrub/scrub land cover. | 1.29105752800456% | 100 |
| Percentage of the upstream watershed area classified as shrub/scrub land cover. | 7.70496436718955% | 100 |
| Percentage of the local catchment classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | .910470409711684% | 100 |
| Percentage of the upstream watershed classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 5.79783852511125% | 100 |
| Percentage of the local catchment area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, low intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, low intensity land use. | .0186744190741526% | 100 |
| Percentage of the local catchment classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | .0169527442254715% | 100 |
| Percentage of the local catchment area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, open space land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, open space land use. | .323016978579937% | 100 |
| Percentage of the local catchment classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | .440771349862259% | 100 |
| Percentage of the local catchment area classified as woody wetland land cover. | .645528764002279% | 100 |
| Percentage of the upstream watershed area classified as woody wetland land cover. | .0312922697999314% | 100 |
| Percentage of the local catchment classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | 5.15933232169954% | 100 |
| Percentage of the upstream watershed classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .262767535494808% | 100 |

#### Landscape Layer: 2011 National Land Cover Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage of the local catchment area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 0% | 100 |
| Percentage of the upstream watershed area classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. | 13.6141562190863% | 100 |
| Percentage of the local catchment classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as area of bedrock, desert pavement, scarps, talus, slides, volcanic materical, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material within a 100-m wide buffer of the NHD stream lines. | 2.93706293706294% | 100 |
| Percentage of the local catchment area classified as evergreen forest land cover. | 96.6014809189292% | 100 |
| Percentage of the upstream watershed area classified as evergreen forest land cover. | 70.6831809096966% | 100 |
| Percentage of the local catchment classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 91.350531107739% | 100 |
| Percentage of the upstream watershed classified as evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 86.4674719220174% | 100 |
| Percentage of the local catchment area classified as row crop land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as row crop land use. | 0% | 100 |
| Percentage of the local catchment classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as crop land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as deciduous forest land cover. | .721473324473134% | 100 |
| Percentage of the upstream watershed area classified as deciduous forest land cover. | .090343811196576% | 100 |
| Percentage of the local catchment classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | 5.76631259484067% | 100 |
| Percentage of the upstream watershed classified as deciduous forest land cover within a 100-m wide buffer of the NHD stream lines. | .322102140283959% | 100 |
| Percentage of the local catchment area classified as grassland/herbaceous land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as grassland/herbaceous land cover. | 3.9160761512527% | 100 |
| Percentage of the local catchment classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as grassland/herbaceous land cover within a 100-m wide buffer of the NHD stream lines. | .979020979020979% | 100 |
| Percentage of the local catchment area classified as pasture/hay land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as pasture/hay land use. | .0171602769870591% | 100 |
| Percentage of the local catchment classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as pasture/hay land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as herbaceous wetland land cover. | 0% | 100 |
| Percentage of the local catchment classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as emergent herbaceous wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as ice/snow land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as ice/snow land cover. | 3.25843377142511% | 100 |
| Percentage of the local catchment classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as ice/snow land cover within a 100-m wide buffer of the NHD stream lines. | 1.6486543759271% | 100 |
| Percentage of the local catchment area classified as mixed deciduous/evergreen forest land cover. | 1.49990506929941% | 100 |
| Percentage of the upstream watershed area classified as mixed deciduous/evergreen forest land cover. | .294752992954192% | 100 |
| Percentage of the local catchment classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | 1.97268588770865% | 100 |
| Percent of the upstream watershed classified as mixed deciduous/evergreen forest land cover within a 100-m wide buffer of the NHD stream lines. | .61877516422971% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as open water land cover. | .0262451295096198% | 100 |
| Percentage of the local catchment classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as open water land cover within a 100-m wide buffer of the NHD stream lines. | .0254291163382072% | 100 |
| Percentage of the local catchment area classified as shrub/scrub land cover. | 1.17714068729827% | 100 |
| Percentage of the upstream watershed area classified as shrub/scrub land cover. | 7.63379968909616% | 100 |
| Percentage of the local catchment classified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | .910470409711684% | 100 |
| Percent of the upstream watershedclassified as shrub/scrub land cover within a 100-m wide buffer of the NHD stream lines. | 5.86988768806951% | 100 |
| Percentage of the local catchment area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, high intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, high intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, low intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, low intensity land use. | .0186744190741526% | 100 |
| Percentage of the local catchment classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, low intensity land use within a 100-m wide buffer of the NHD stream lines. | .0169527442254715% | 100 |
| Percentage of the local catchment area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, medium intensity land use. | 0% | 100 |
| Percentage of the local catchment classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, medium intensity land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the local catchment area classified as developed, open space land use. | 0% | 100 |
| Percentage of the upstream watershed area classified as developed, open space land use. | .323016978579937% | 100 |
| Percentage of the local catchment classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percentage of the upstream watershed classified as developed, open space land use within a 100-m wide buffer of the NHD stream lines. | .440771349862259% | 100 |
| Percentage of the local catchment area classified as woody wetland land cover. | 0% | 100 |
| Percentage of the upstream watershed area classified as woody wetland land cover. | .124159651141663% | 100 |
| Percentage of the local catchment classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | 0% | 100 |
| Percent of the upstream watershed classified as woody wetland land cover within a 100-m wide buffer of the NHD stream lines. | .673871582962492% | 100 |

#### Landscape Layer: Nonnative LANDFIRE Vegetation
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the local catchment. | 0% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the total upstream watershed. | .642500958956655% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the riparian mask of the local catchment. | 0% | 100 |
| Percentage nonnative vegetation landcover type reclassed from LANDFIRE Existing Vegetation Type (EVT) within the riparian mask of the total upstream watershed. | .627251536342445% | 100 |

#### Landscape Layer: Pesticide
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Total pesticides per square kilometer within the local catchment (kilograms/square kilometer). | 0kg/km² | 100 |
| Total pesticides per square kilometer within the total upstream watershed (kilograms/square kilometer). | 0kg/km² | 100 |

#### Landscape Layer: PRISM Data 2008-09
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean 2008 precipitation (mm) within the local catchment. | 792.84771469mm | 100 |
| Mean 2008 precipitation (mm) within the total upstream watershed. | 1214.72727726mm | 100 |
| Mean 2009 precipitation (mm) within the local catchment. | 1058.93075445mm | 100 |
| Mean 2009 precipitation (mm) within the total upstream watershed. | 1472.24516547mm | 100 |
| Mean 2008 air temperature (Celcius) within the local catchment. | 7.83102128193°C | 100 |
| Mean 2008 air temperature (Celcius) within the total upstream watershed. | 4.91159862035°C | 100 |
| Mean 2009 air temperature (Celcius) within the local catchment. | 8.70481992569°C | 100 |
| Mean 2009 air temperature (Celcius) within the total upstream watershed. | 5.79357596582°C | 100 |

#### Landscape Layer: PRISM Normals Data
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| 30-year average annual normal precipitation (mm) within the local catchment. | 1135.31072626mm | 100 |
| 30-year average annual normal precipitation (mm) within the upstream watershed. | 1574.28425462mm | 100 |
| 30-year average annual normal maximum air temperature (Celcius) within the local catchment. | 13.1880910773°C | 100 |
| 30-year average annual normal maximum air temperature (Celcius) within the upstream watershed. | 9.3704917554°C | 100 |
| 30-year average annual normal mean air temperature (Celcius) within the local catchment. | 8.68293559937°C | 100 |
| 30-year average annual normal mean air temperature (Celcius) within the upstream watershed. | 5.16514524453°C | 100 |
| 30-year average annual normal minimum air temperature (Celcius) within the local catchment. | 4.1719242685°C | 100 |
| 30-year average annual normal minimum air temperature (Celcius) within the upstream watershed. | .954895208709°C | 100 |

#### Landscape Layer: Reference Stream Temperature Predictions
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Predicted annual stream temperature (Celcius) for year 2008. | 8.09534055444°C |  |
| Predicted annual stream temperature (Celcius) for year 2009. | 8.0548024219°C |  |
| Predicted annual stream temperature (Celcius) for year 2013. | 8.16391329403°C |  |
| Predicted annual stream temperature (Celcius) for year 2014. | 8.86627182366°C |  |
| Predicted summer stream temperature (Celcius) for year 2008. | 13.4068878161°C |  |
| Predicted summer stream temperature (Celcius) for year 2009. | 13.9892171145°C |  |
| Predicted summer stream temperature (Celcius) for year 2013. | 13.9198268412°C |  |
| Predicted summer stream temperature (Celcius) for year 2014. | 14.2707118015°C |  |
| Predicted winter stream temperature (Celcius) for year 2008. | 4.1540904119°C |  |
| Predicted winter stream temperature (Celcius) for year 2009. | 4.38654265717°C |  |
| Predicted winter stream temperature (Celcius) for year 2013. | 4.37222816586°C |  |
| Predicted winter stream temperature (Celcius) for year 2014. | 4.24042778°C |  |

#### Landscape Layer: 2010 US Census Road Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Average density of roads per square kilometer within the local catchment (kilometer of road/square kilometer). | .162701556380292km/km² | 100 |
| Average density of roads per square kilometer within the total upstream watershed (kilometer of road/square kilometer). | .0964538293909313km/km² | 100 |
| Average density of roads per square kilometer within the riparian mask of the local catchment (kilometer of road/square kilometer). | 0km/km² | 100 |
| Average density of roads per square kilometer within the riparian mask of the total upstream watershed (kilometer of road/square kilometer). | .0867531528103412km/km² | 100 |

#### Landscape Layer: Road and Stream Intersections
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of rdstrcrs values (crossings / square kilometer) within the local catchment. | 0crossings/km² | 100 |
| Mean of rdstrcrs values (crossings / square kilometer) within the total upstream watershed. | .0056079336559crossings/km² | 100 |
| Rdstrcrs values multiplied by NHD slope value from elevslope.dbf (crossings * slope / square kilometer) within the local catchment. | 0crossings * slope/km² | 100 |
| Rdstrcrs values multiplied by NHD slope value from elevslope.dbf (crossings * slope / square kilometer) within the total upstream watershed. | .000292970735559crossings * slope/km² | 100 |

#### Landscape Layer: Runoff
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all runoff values within the local catchment. | 1575mm | 100 |
| Mean of all runoff values within the upstream watershed. | 1585.8703843902mm | 100 |

#### Landscape Layer: State Soil Geographic Database
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all clay values within the local catchment. | 16.9495728118474% | 100 |
| Mean of all clay values within the upstream watershed. | 14.7577501362728% | 100 |
| Mean of all sand values within the local catchment. | 36.9839415226884% | 100 |
| Mean of all sand values within the upstream watershed. | 51.0040726384431% | 100 |
| Mean of all organic matter values within the local catchment. | 1.23218340611354% | 100 |
| Mean of all organic matter values within the upstream watershed. | 1.71094656287976% | 76.8351402096 |
| Mean of all permeability of soils values within the local catchment. | 3.79752610594266cm/hour | 100 |
| Mean of all permeability of soils values within the upstream watershed. | 4.26550802033658cm/hour | 76.8351402096 |
| Mean of all depth to bedrock of soils values within the local catchment. | 111.741674577558cm | 100 |
| Mean of all depth to bedrock of soils values within the upstream watershed. | 85.4435758483427cm | 76.8351402096 |
| Mean of all seasonal water table depth of soils values within the local catchment. | 180.915412948548cm | 100 |
| Mean of all seasonal water table depth of soils values within the upstream watershed. | 182.369055886178cm | 76.8351402096 |

#### Landscape Layer: 2010 US Census Housing Unit and Population Density
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean of all housing units per square kilometer values within the local catchment. | 2.37879685515474housing units/km² | 100 |
| Mean of all housing units per square kilometer values within the upstream watershed. | 2.02547570364706housing units/km² | 100 |
| Mean of all housing units per square kilometer within the local catchment within a 100-m wide buffer of the NHD stream lines. | 2.72826706578149housing units/km² | 100 |
| Mean of all housing units per square kilometer within the upstream watershed within a 100-m wide buffer of the NHD stream lines. | 2.16261078840432housing units/km² | 100 |
| Mean of all 2010 population per square kilometer values within the local catchment. | 5.10921317717866people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the upstream watershed. | 3.38870089597844people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the local catchment within a 100-m wide buffer of the NHD stream lines. | 5.61240491312595people/km² | 100 |
| Mean of all 2010 population per square kilometer values within the upstream watershed within a 100-m wide buffer of the NHD stream lines. | 3.79032108265734people/km² | 100 |

#### Landscape Layer: Wetness Index
| Metric Description | Metric Value | AOI Percent Covered |
|---|---|---|
| Mean Composite Topographic Index (CTI) [Wetness Index] within the local catchment. | 277.364344028859 | 100 |
| Mean Composite Topographic Index (CTI) [Wetness Index] within the upstream watershed. | 243.670850745967 | 100 |



[DEBUG] Hover event: mouseover, feature: {'id': '2', 'type': 'Feature', 'properties': {'type': 'hydrolocation', 'source': 'nwissite', 'sourceName': 'NWIS Surface Water Sites', 'identifier': 'USGS-12048000', 'name': 'DUNGENESS RIVER NEAR SEQUIM, WA', 'uri': 'https://waterdata.usgs.gov/monitoring-location/12048000', 'comid': '23997388', 'reachcode': '17110020000165', 'measure': '68.5008971902', 'navigation': 'https://api.water.usgs.gov/nldi/linked-data/nwissite/USGS-12048000/navigation'}, 'geometry': {'type': 'Point', 'coordinates': [-123.13267523071997, 48.01425832739993]}}, id: 2, coords: None
